# Finetuning ruGPT3Xl example
We test this pipeline on `torch 1.7.1+cu110, cuda 11.0, triton 1.0.0, deepspeed 0.7.4`, `A100 GPU` and it takes around `32GB GPU memory`.

## Install env

In [1]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

In [3]:
!rm -rf apex

In [ ]:
%%bash
git clone https://github.com/qywu/apex
cd apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
!pip install triton==1.0.0

In [ ]:
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.7.4

In [7]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [YES] ...... [OKAY]
cpu_adagrad ............ [NO] ....... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
sparse_attn ............ [YES] ...... [OKAY]
transformer ............ [NO] ....... [OKAY]
stochastic_transformer . [NO] ....... [OKAY]
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not fou

Test, that deepspeed was installed correctly.

In [14]:
import deepspeed.ops.sparse_attention.sparse_attn_op

In [9]:
!rm -rf ru-gpts

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

In [ ]:
!pip install transformers==4.24.0

In [ ]:
!pip install timm==0.3.2

In [13]:
!cp ru-gpts/src_utils/trainer_pt_utils.py /usr/local/lib/python3.8/dist-packages/transformers/trainer_pt_utils.py

In [ ]:
!cp ru-gpts/src_utils/_amp_state.py /usr/local/lib/python3.8/dist-packages/apex/amp/_amp_state.py

## Prepare data
Download example data.

In [ ]:
!wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
!wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

We should pass to our model list of text files as following:

In [2]:
%%writefile train.list
train.txt

Writing train.list


In [3]:
%%writefile valid.list
valid.txt

Writing valid.list


## Start finetune
We configure training script at `scripts/deepspeed_gpt3_xl_finetune.sh`

```bash
%%bash

# Model parallel size
MP_SIZE=1
# Change for multinode config
NUM_GPUS_PER_WORKER=1

gpt_options=" \
       --train-data-path examples/train.list \
       --test-data-path examples/valid.list \
       --load-huggingface sberbank-ai/rugpt3xl \
       --logging-dir=examples/log/ \
       --save examples/model \
       --save-interval 200 \
       --model-parallel-size ${MP_SIZE} \
       --num-layers 24 \
       --hidden-size 2048 \
       --num-attention-heads 16 \
       --batch-size 1 \
       --seq-length 2048 \
       --max-position-embeddings 2048 \
       --train-iters 1000 \
       --distributed-backend nccl \
       --lr 0.0002 \
       --lr-decay-style cosine \
       --weight-decay 1e-2 \
       --warmup .01 \
       --log-interval 50 \
       --fp16 \
       --checkpoint-activations \
       --deepspeed-activation-checkpointing \
       --sparse-mode alternating \
       --deepspeed \
       --deepspeed_config src/deepspeed_config/gpt3_xl_sparse_2048.json \
"

run_cmd="USE_DEEPSPEED=1 python -m torch.distributed.launch --nproc_per_node=${NUM_GPUS_PER_WORKER} pretrain_gpt3.py $@ ${gpt_options}"
echo "${run_cmd}"
eval "${run_cmd}"

set +x
```

Here
* `train-data-path` - train data path
* `test-data-path` - test data path
* `load-huggingface` - model with we want to finetune

and other <i>standart</i> tuning parameters.

**Note!** `batch-size 1` because we use only 32 GB of GPU memory.

### Run script from content of repo root!!!
`sh scripts/deepspeed_gpt3_xl_finetune.sh`

After run training script we can see progress on tensorboard: 

In [19]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir log/

## Test model

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append("../../ru-gpts/")

In [3]:
import os
os.environ["USE_DEEPSPEED"] = "1"

In [4]:
from src.xl_wrapper import RuGPT3XL

2022-12-06 21:35:45.765088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Note! seq_len is max sequence length for generation used in generation process. Max avialable seq_len is 2048 (in tokens).
Also inference takes around 10 Gb GPU memory.

In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
from src import mpu

In [7]:
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "5000"

In [ ]:
# gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)
gpt = RuGPT3XL.from_pretrained(
    "sberbank-ai/rugpt3xl",
    weights_path="model/400/mp_rank_00_model_states.pt",
    seq_len=2048
)
# model parallel group is not initialized - если не подключена gpu

In [12]:
def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

In [13]:
res = gpt.generate(
    "<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: ",
    max_length=512,
    repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1
)

In [16]:
res[0][:1000]

'<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: \xa0\xa0\xa0 | - Человек это высшая ступень развития живых организмов на земле, субъект общественно-исторической деятельностии культуры; — человек прочно связан с обществом как со своим главным агентом социализации детей раннего возраста. Согласно антропосоциогенеза структура человеческой личности определяется четырьмя основными составляющими : соматологической(составление карты больного), психической («мысль изреченная есть закон законов»), социальной активности поведения ребенка в момент рождения или вскоре после него.(А какие же качества лично вам кажутся самыми важными? Думается мне что автор хотел указать значение игровой игры для формирования здорового молодого организма.) И я полностью разделяю мнение известного литератора В.-Г..Белинского о том,,что игра является основным фактором обновления основных жизненных сил человеческого тела... Действительно ли так бывает? У каждого 